In [1]:
## todo: 
# 解决测评函数不能引入cv的问题
# 建立cv流程进行调参
# 清理异常点(小于0)

In [2]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier
from sklearn.metrics import confusion_matrix

import itertools
import matplotlib.pyplot as plt
%matplotlib inline

train = pd.read_table('train/subsidy_train.txt',sep=',',header=-1)
train.columns = ['id','label']
test = pd.read_table('test/studentID_test.txt',sep=',',header=-1)
test.columns = ['id']
test['label'] = np.nan

#train_test = pd.concat([train,test])
train_test=train

del train
del test

score_train = pd.read_table('train/score_train.txt',sep=',',header=-1)
score_train.columns = ['id','college','rank']
score_test = pd.read_table('test/score_test.txt',sep=',',header=-1)
score_test.columns = ['id','college','rank']
score_train_test = pd.concat([score_train,score_test])

college = pd.DataFrame(score_train_test.groupby(['college'])['rank'].max())
college.to_csv('input/college.csv',index=True)
college = pd.read_csv('input/college.csv')
college.columns = ['college','total_people']

score_train_test = pd.merge(score_train_test, college, how='left',on='college')
score_train_test['rank_percent'] = score_train_test['rank']/score_train_test['total_people']
train_test = pd.merge(train_test,score_train_test,how='left',on='id')

card_train = pd.read_table('train/card_train.txt',sep=',',header=-1)
card_train.columns = ['id','pos','place','consume','time','price','rest']
#card_test = pd.read_table('test/card_test.txt',sep=',',header=-1)
#card_test.columns = ['id','pos','place','consume','time','price','rest']
#
#card_train_test = pd.concat([card_train,card_test])
card_train_test=card_train
print "Read OK!"

##release memery
del card_train
#del card_test

#card_train_test=card_train_test.drop_duplicates()

#card_train_test.drop(['place','time'],axis=1,inplace=True)

card_train_test.shape

card_train_test.head()

#card_train_test['pos'].value_counts()

for var in ['地点21','地点829','地点818','地点213','地点72','地点283','地点91','地点245','地点65','地点161','地点996','地点277','地点842','地点75','地点263','地点840']:
    
    place = card_train_test[card_train_test.place == var]
    
    feature_p = pd.DataFrame(place.groupby(['id'])['pos'].count())
    feature_p['%s_sum'%var]=place.groupby(['id'])['price'].sum()
    feature_p['%s_avg'%var]=place.groupby(['id'])['price'].mean()
    feature_p['%s_max'%var]=place.groupby(['id'])['price'].max()
    feature_p['%s_min'%var]=place.groupby(['id'])['price'].min()
    feature_p['%s_median'%var]=place.groupby(['id'])['price'].median()
    
    del place
    feature_p.to_csv('input/card_%sfeature.csv'%var,index=True)
    feature_p=pd.read_csv('input/card_%sfeature.csv'%var)
    feature_p=feature_p.rename(columns={'pos' : '%s_count'%var})
    
    train_test = pd.merge(train_test, feature_p, how='left',on='id') 
    del feature_p

card_train_test.time = pd.to_datetime(card_train_test.time, format='%Y/%m/%d %H:%M:%S')
card_train_test['month'] = card_train_test.time.dt.month
card_train_test['weekday'] = card_train_test.time.dt.weekday
card_train_test['days_in_month'] = card_train_test.time.dt.day
card_train_test['hour'] = card_train_test.time.dt.hour
#card_train_test.drop(['place','time'],axis=1,inplace=True)

#card_train_test.drop(['place','consume','time'],axis=1,inplace=True)

# tips：
# - month 分月统**消费**情况 //（特别注意开学和学期结束（补助到账） 
# - day 统计充钱时间 //工薪家庭定期发放生活费，猜测与补助成负相关
# - hour 统计消费情况 
# - week 分为周末于工作日统计
'''
for m in range(1,13):
    for d in range(1,32):
        for h in range(0,24):
            for w in range(0,7):
                feature=card_train_test[(card_train_test.month == m)&(card_train_test.weekday == w)&(card_train_test.days_in_month == d)&(card_train_test.hour == h)] 
                if feature.empty:
                    pass
                else:
                    card = pd.DataFrame(feature.groupby(['id'])['pos'].count())
                    
                    card['price_sum%d_%d_%d_%d'%(m,d,h,w)] = feature.groupby(['id'])['price'].sum()
                    card['price_avg%d_%d_%d_%d'%(m,d,h,w)] = feature.groupby(['id'])['price'].mean()
                    card['price_max%d_%d_%d_%d'%(m,d,h,w)] = feature.groupby(['id'])['price'].max()
                    card['price_min%d_%d_%d_%d'%(m,d,h,w)] = feature.groupby(['id'])['price'].min()
                    card['price_median%d_%d_%d_%d'%(m,d,h,w)] = feature.groupby(['id'])['price'].median()

                    card['rest_sum%d_%d_%d_%d'%(m,d,h,w)] = feature.groupby(['id'])['rest'].sum()
                    card['rest_avg%d_%d_%d_%d'%(m,d,h,w)] = feature.groupby(['id'])['rest'].mean()
                    card['rest_max%d_%d_%d_%d'%(m,d,h,w)] = feature.groupby(['id'])['rest'].max()
                    card['rest_min%d_%d_%d_%d'%(m,d,h,w)] = feature.groupby(['id'])['rest'].min()
                    card['rest_median%d_%d_%d_%d'%(m,d,h,w)] = feature.groupby(['id'])['rest'].median()
                    
                    card.to_csv('input/feature%d_%d_%d_%d.csv'%(m,d,h,w),index=True)
                    card = pd.read_csv('input/feature%d_%d_%d_%d.csv'%(m,d,h,w)) 
                    card=card.rename(columns={'pos' : 'count%d_%d_%d_%d'%(m,d,h,w)}) 
                    train_test = pd.merge(train_test, card, how='left',on='id')
                    del card
'''

for m in range(1,13):
    feature=card_train_test[(card_train_test.month == m)&(card_train_test.pos=='POS消费')] 
    if feature.empty:
        pass
    else:
        card = pd.DataFrame(feature.groupby(['id'])['pos'].count())
                    
        card['price_sumM%d'%m] = feature.groupby(['id'])['price'].sum()
        card['price_avgM%d'%m] = feature.groupby(['id'])['price'].mean()
        card['price_maxM%d'%m] = feature.groupby(['id'])['price'].max()
        card['price_minM%d'%m] = feature.groupby(['id'])['price'].min()
        card['price_medianM%d'%m] = feature.groupby(['id'])['price'].median()

        card['rest_sumM%d'%m] = feature.groupby(['id'])['rest'].sum()
        card['rest_avgM%d'%m] = feature.groupby(['id'])['rest'].mean()
        card['rest_maxM%d'%m] = feature.groupby(['id'])['rest'].max()
        card['rest_minM%d'%m] = feature.groupby(['id'])['rest'].min()
        card['rest_medianM%d'%m] = feature.groupby(['id'])['rest'].median() 

        del feature
        
        card.to_csv('input/featureM%d.csv'%m,index=True)
        card = pd.read_csv('input/featureM%d.csv'%m) 
        card=card.rename(columns={'pos' : 'countM%d'%m}) 
        train_test = pd.merge(train_test, card, how='left',on='id')
        del card
    

'''
for d in range(1,32):
    feature=card_train_test[(card_train_test.days_in_month == d)&((card_train_test.pos=='卡充值')|(card_train_test.pos=='圈存转账'))] 
    if feature.empty:
        pass
    else:
        card = pd.DataFrame(feature.groupby(['id'])['pos'].count())
                    
        card['price_sumD%d'%d] = feature.groupby(['id'])['price'].sum()
        card['price_avgD%d'%d] = feature.groupby(['id'])['price'].mean()
        card['price_maxD%d'%d] = feature.groupby(['id'])['price'].max()
        card['price_minD%d'%d] = feature.groupby(['id'])['price'].min()
        card['price_medianD%d'%d] = feature.groupby(['id'])['price'].median()

        card['rest_sumD%d'%d] = feature.groupby(['id'])['rest'].sum()
        card['rest_avgD%d'%d] = feature.groupby(['id'])['rest'].mean()
        card['rest_maxD%d'%d] = feature.groupby(['id'])['rest'].max()
        card['rest_minD%d'%d] = feature.groupby(['id'])['rest'].min()
        card['rest_medianD%d'%d] = feature.groupby(['id'])['rest'].median()
        del feature
            
        card.to_csv('input/featureD%d.csv'%d,index=True)
        card = pd.read_csv('input/featureD%d.csv'%d) 
        card=card.rename(columns={'pos' : 'countD%d'%d}) 
        train_test = pd.merge(train_test, card, how='left',on='id')
        del card
        
'''

## hours <7|7|8|9|17|18|19|19>

feature=card_train_test[(card_train_test.hour<7)&(card_train_test.pos=='POS消费')] 
if feature.empty:
    pass
else:
    card = pd.DataFrame(feature.groupby(['id'])['pos'].count())
                    
    card['price_sumH7-'] = feature.groupby(['id'])['price'].sum()
    card['price_avgH7-'] = feature.groupby(['id'])['price'].mean()
    card['price_maxH7-'] = feature.groupby(['id'])['price'].max()
    card['price_minH7-'] = feature.groupby(['id'])['price'].min()
    card['price_medianH7-'] = feature.groupby(['id'])['price'].median()

    card['rest_sumH7-'] = feature.groupby(['id'])['rest'].sum()
    card['rest_avgH7-'] = feature.groupby(['id'])['rest'].mean()
    card['rest_maxH7-'] = feature.groupby(['id'])['rest'].max()
    card['rest_minH7-'] = feature.groupby(['id'])['rest'].min()
    card['rest_medianH7-'] = feature.groupby(['id'])['rest'].median()

    del feature
            
    card.to_csv('input/featureH7-.csv',index=True)
    card = pd.read_csv('input/featureH7-.csv') 
    card=card.rename(columns={'pos' : 'countH7-'}) 
    train_test = pd.merge(train_test, card, how='left',on='id')
    del card


h=7
feature=card_train_test[(card_train_test.hour== h)&(card_train_test.pos=='POS消费')] 
if feature.empty:
    pass
else:
    card = pd.DataFrame(feature.groupby(['id'])['pos'].count())
                    
    card['price_sumH%d'%h] = feature.groupby(['id'])['price'].sum()
    card['price_avgH%d'%h] = feature.groupby(['id'])['price'].mean()
    card['price_maxH%d'%h] = feature.groupby(['id'])['price'].max()
    card['price_minH%d'%h] = feature.groupby(['id'])['price'].min()
    card['price_medianH%d'%h] = feature.groupby(['id'])['price'].median()

    card['rest_sumH%d'%h] = feature.groupby(['id'])['rest'].sum()
    card['rest_avgH%d'%h] = feature.groupby(['id'])['rest'].mean()
    card['rest_maxH%d'%h] = feature.groupby(['id'])['rest'].max()
    card['rest_minH%d'%h] = feature.groupby(['id'])['rest'].min()
    card['rest_medianH%d'%h] = feature.groupby(['id'])['rest'].median()

    del feature
            
    card.to_csv('input/featureH%d.csv'%h,index=True)
    card = pd.read_csv('input/featureH%d.csv'%h) 
    card=card.rename(columns={'pos' : 'countH%d'%h}) 
    train_test = pd.merge(train_test, card, how='left',on='id')
    del card

h=8
feature=card_train_test[(card_train_test.hour== h)&(card_train_test.pos=='POS消费')] 
if feature.empty:
    pass
else:
    card = pd.DataFrame(feature.groupby(['id'])['pos'].count())
                    
    card['price_sumH%d'%h] = feature.groupby(['id'])['price'].sum()
    card['price_avgH%d'%h] = feature.groupby(['id'])['price'].mean()
    card['price_maxH%d'%h] = feature.groupby(['id'])['price'].max()
    card['price_minH%d'%h] = feature.groupby(['id'])['price'].min()
    card['price_medianH%d'%h] = feature.groupby(['id'])['price'].median()

    card['rest_sumH%d'%h] = feature.groupby(['id'])['rest'].sum()
    card['rest_avgH%d'%h] = feature.groupby(['id'])['rest'].mean()
    card['rest_maxH%d'%h] = feature.groupby(['id'])['rest'].max()
    card['rest_minH%d'%h] = feature.groupby(['id'])['rest'].min()
    card['rest_medianH%d'%h] = feature.groupby(['id'])['rest'].median()

    del feature
            
    card.to_csv('input/featureH%d.csv'%h,index=True)
    card = pd.read_csv('input/featureH%d.csv'%h) 
    card=card.rename(columns={'pos' : 'countH%d'%h}) 
    train_test = pd.merge(train_test, card, how='left',on='id')
    del card
                
h=9
feature=card_train_test[(card_train_test.hour== h)&(card_train_test.pos=='POS消费')] 
if feature.empty:
    pass
else:
    card = pd.DataFrame(feature.groupby(['id'])['pos'].count())
                    
    card['price_sumH%d'%h] = feature.groupby(['id'])['price'].sum()
    card['price_avgH%d'%h] = feature.groupby(['id'])['price'].mean()
    card['price_maxH%d'%h] = feature.groupby(['id'])['price'].max()
    card['price_minH%d'%h] = feature.groupby(['id'])['price'].min()
    card['price_medianH%d'%h] = feature.groupby(['id'])['price'].median()

    card['rest_sumH%d'%h] = feature.groupby(['id'])['rest'].sum()
    card['rest_avgH%d'%h] = feature.groupby(['id'])['rest'].mean()
    card['rest_maxH%d'%h] = feature.groupby(['id'])['rest'].max()
    card['rest_minH%d'%h] = feature.groupby(['id'])['rest'].min()
    card['rest_medianH%d'%h] = feature.groupby(['id'])['rest'].median()

    del feature
            
    card.to_csv('input/featureH%d.csv'%h,index=True)
    card = pd.read_csv('input/featureH%d.csv'%h) 
    card=card.rename(columns={'pos' : 'countH%d'%h}) 
    train_test = pd.merge(train_test, card, how='left',on='id')
    del card
                
h=17
feature=card_train_test[(card_train_test.hour== h)&(card_train_test.pos=='POS消费')] 
if feature.empty:
    pass
else:
    card = pd.DataFrame(feature.groupby(['id'])['pos'].count())
                    
    card['price_sumH%d'%h] = feature.groupby(['id'])['price'].sum()
    card['price_avgH%d'%h] = feature.groupby(['id'])['price'].mean()
    card['price_maxH%d'%h] = feature.groupby(['id'])['price'].max()
    card['price_minH%d'%h] = feature.groupby(['id'])['price'].min()
    card['price_medianH%d'%h] = feature.groupby(['id'])['price'].median()

    card['rest_sumH%d'%h] = feature.groupby(['id'])['rest'].sum()
    card['rest_avgH%d'%h] = feature.groupby(['id'])['rest'].mean()
    card['rest_maxH%d'%h] = feature.groupby(['id'])['rest'].max()
    card['rest_minH%d'%h] = feature.groupby(['id'])['rest'].min()
    card['rest_medianH%d'%h] = feature.groupby(['id'])['rest'].median()

    del feature
            
    card.to_csv('input/featureH%d.csv'%h,index=True)
    card = pd.read_csv('input/featureH%d.csv'%h) 
    card=card.rename(columns={'pos' : 'countH%d'%h}) 
    train_test = pd.merge(train_test, card, how='left',on='id')
    del card
                
h=18
feature=card_train_test[(card_train_test.hour== h)&(card_train_test.pos=='POS消费')] 
if feature.empty:
    pass
else:
    card = pd.DataFrame(feature.groupby(['id'])['pos'].count())
                    
    card['price_sumH%d'%h] = feature.groupby(['id'])['price'].sum()
    card['price_avgH%d'%h] = feature.groupby(['id'])['price'].mean()
    card['price_maxH%d'%h] = feature.groupby(['id'])['price'].max()
    card['price_minH%d'%h] = feature.groupby(['id'])['price'].min()
    card['price_medianH%d'%h] = feature.groupby(['id'])['price'].median()

    card['rest_sumH%d'%h] = feature.groupby(['id'])['rest'].sum()
    card['rest_avgH%d'%h] = feature.groupby(['id'])['rest'].mean()
    card['rest_maxH%d'%h] = feature.groupby(['id'])['rest'].max()
    card['rest_minH%d'%h] = feature.groupby(['id'])['rest'].min()
    card['rest_medianH%d'%h] = feature.groupby(['id'])['rest'].median()

    del feature
            
    card.to_csv('input/featureH%d.csv'%h,index=True)
    card = pd.read_csv('input/featureH%d.csv'%h) 
    card=card.rename(columns={'pos' : 'countH%d'%h}) 
    train_test = pd.merge(train_test, card, how='left',on='id')
    del card
                
h=19
feature=card_train_test[(card_train_test.hour== h)&(card_train_test.pos=='POS消费')] 
if feature.empty:
    pass
else:
    card = pd.DataFrame(feature.groupby(['id'])['pos'].count())
                    
    card['price_sumH%d'%h] = feature.groupby(['id'])['price'].sum()
    card['price_avgH%d'%h] = feature.groupby(['id'])['price'].mean()
    card['price_maxH%d'%h] = feature.groupby(['id'])['price'].max()
    card['price_minH%d'%h] = feature.groupby(['id'])['price'].min()
    card['price_medianH%d'%h] = feature.groupby(['id'])['price'].median()

    card['rest_sumH%d'%h] = feature.groupby(['id'])['rest'].sum()
    card['rest_avgH%d'%h] = feature.groupby(['id'])['rest'].mean()
    card['rest_maxH%d'%h] = feature.groupby(['id'])['rest'].max()
    card['rest_minH%d'%h] = feature.groupby(['id'])['rest'].min()
    card['rest_medianH%d'%h] = feature.groupby(['id'])['rest'].median()

    del feature
            
    card.to_csv('input/featureH%d.csv'%h,index=True)
    card = pd.read_csv('input/featureH%d.csv'%h) 
    card=card.rename(columns={'pos' : 'countH%d'%h}) 
    train_test = pd.merge(train_test, card, how='left',on='id')
    del card
                

feature=card_train_test[(card_train_test.hour>19)&(card_train_test.pos=='POS消费')] 
if feature.empty:
    pass
else:
    card = pd.DataFrame(feature.groupby(['id'])['pos'].count())
                    
    card['price_sumH19+'] = feature.groupby(['id'])['price'].sum()
    card['price_avgH19+'] = feature.groupby(['id'])['price'].mean()
    card['price_maxH19+'] = feature.groupby(['id'])['price'].max()
    card['price_minH19+'] = feature.groupby(['id'])['price'].min()
    card['price_medianH19+'] = feature.groupby(['id'])['price'].median()

    card['rest_sumH19+'] = feature.groupby(['id'])['rest'].sum()
    card['rest_avgH19+'] = feature.groupby(['id'])['rest'].mean()
    card['rest_maxH19+'] = feature.groupby(['id'])['rest'].max()
    card['rest_minH19+'] = feature.groupby(['id'])['rest'].min()
    card['rest_medianH19+'] = feature.groupby(['id'])['rest'].median()

    del feature
            
    card.to_csv('input/featureH19+.csv',index=True)
    card = pd.read_csv('input/featureH19+.csv') 
    card=card.rename(columns={'pos' : 'countH19+'}) 
    train_test = pd.merge(train_test, card, how='left',on='id')
    del card

## week

feature=card_train_test[(card_train_test.weekday<5)&(card_train_test.pos=='POS消费')] 
if feature.empty:
    pass
else:
    card = pd.DataFrame(feature.groupby(['id'])['pos'].count())
                    
    card['price_sumWD'] = feature.groupby(['id'])['price'].sum()
    card['price_avgWD'] = feature.groupby(['id'])['price'].mean()
    card['price_maxWD'] = feature.groupby(['id'])['price'].max()
    card['price_minWD'] = feature.groupby(['id'])['price'].min()
    card['price_medianWD'] = feature.groupby(['id'])['price'].median()

    card['rest_sumWD'] = feature.groupby(['id'])['rest'].sum()
    card['rest_avgWD'] = feature.groupby(['id'])['rest'].mean()
    card['rest_maxWD'] = feature.groupby(['id'])['rest'].max()
    card['rest_minWD'] = feature.groupby(['id'])['rest'].min()
    card['rest_medianWD'] = feature.groupby(['id'])['rest'].median()

    del feature
            
    card.to_csv('input/featureWD.csv',index=True)
    card = pd.read_csv('input/featureWD.csv') 
    card=card.rename(columns={'pos' : 'countWD'}) 
    train_test = pd.merge(train_test, card, how='left',on='id')
    del card


feature=card_train_test[(card_train_test.weekday>=5)&(card_train_test.pos=='POS消费')] 
if feature.empty:
    pass
else:
    card = pd.DataFrame(feature.groupby(['id'])['pos'].count())
                    
    card['price_sumWE'] = feature.groupby(['id'])['price'].sum()
    card['price_avgWE'] = feature.groupby(['id'])['price'].mean()
    card['price_maxWE'] = feature.groupby(['id'])['price'].max()
    card['price_minWE'] = feature.groupby(['id'])['price'].min()
    card['price_medianWE'] = feature.groupby(['id'])['price'].median()

    card['rest_sumWE'] = feature.groupby(['id'])['rest'].sum()
    card['rest_avgWE'] = feature.groupby(['id'])['rest'].mean()
    card['rest_maxWE'] = feature.groupby(['id'])['rest'].max()
    card['rest_minWE'] = feature.groupby(['id'])['rest'].min()
    card['rest_medianWE'] = feature.groupby(['id'])['rest'].median()

    del feature
            
    card.to_csv('input/featureWE.csv',index=True)
    card = pd.read_csv('input/featureWE.csv') 
    card=card.rename(columns={'pos' : 'countWE'}) 
    train_test = pd.merge(train_test, card, how='left',on='id')
    del card


##这里的consume是有问题的，consume一列包含刷卡和消费

## bash feature
card = pd.DataFrame(card_train_test.groupby(['id'])['pos'].count())

card['price_sum'] = card_train_test.groupby(['id'])['price'].sum()
card['price_avg'] = card_train_test.groupby(['id'])['price'].mean()
#
card['price_max'] = card_train_test.groupby(['id'])['price'].max()
card['price_min'] = card_train_test.groupby(['id'])['price'].min()
card['price_median'] = card_train_test.groupby(['id'])['price'].median()

card['rest_sum'] = card_train_test.groupby(['id'])['rest'].sum()
card['rest_avg'] = card_train_test.groupby(['id'])['rest'].mean()
card['rest_max'] = card_train_test.groupby(['id'])['rest'].max()
card['rest_min'] = card_train_test.groupby(['id'])['rest'].min()
card['rest_median'] = card_train_test.groupby(['id'])['rest'].median()

card.to_csv('input/card_bashfeature.csv',index=True)
card = pd.read_csv('input/card_bashfeature.csv') 
card=card.rename(columns={'pos' : 'price_count'}) 

train_test = pd.merge(train_test, card, how='left',on='id') #2512
del card

'''
## bad feature
## recharge feature
recharge=card_train_test[(card_train_test.pos=='卡充值') | (card_train_test.pos=='圈存转账') ]

card_recharge = pd.DataFrame(recharge.groupby(['id'])['pos'].count())

card_recharge['recharge_sum']=recharge.groupby(['id'])['price'].sum()
card_recharge['recharge_avg']=recharge.groupby(['id'])['price'].mean()
#
card_recharge['recharge_max']=recharge.groupby(['id'])['price'].max()
card_recharge['recharge_min']=recharge.groupby(['id'])['price'].min()
#
card_recharge['recharge_median']=recharge.groupby(['id'])['price'].median() #2330


del recharge
card_recharge.to_csv('input/card_rechargefeature.csv',index=True)
card_recharge = pd.read_csv('input/card_rechargefeature.csv') 
card_recharge=card_recharge.rename(columns={'pos' : 'recharge_count'}) 
train_test = pd.merge(train_test, card_recharge, how='left',on='id') 
del card_recharge

## 支付领取项
zhifu = card_train_test[card_train_test.pos=='支付领取']
card_zhifu= pd.DataFrame(zhifu.groupby(['id'])['pos'].count())

card_zhifu['zhifu_sum'] = zhifu.groupby(['id'])['price'].sum()
card_zhifu['zhifu_avg'] = zhifu.groupby(['id'])['price'].mean()
#
card_zhifu['zhifu_max'] = zhifu.groupby(['id'])['price'].max()
card_zhifu['zhifu_min'] = zhifu.groupby(['id'])['price'].min()
#
card_zhifu['zhifu_median'] = zhifu.groupby(['id'])['price'].median()

del zhifu
card_zhifu.to_csv('input/card_zhifufeature.csv',index=True)
card_zhifu = pd.read_csv('input/card_zhifufeature.csv') 
card_zhifu=card_zhifu.rename(columns={'pos' : 'zhifu_count'}) 


train_test = pd.merge(train_test, card_zhifu, how='left',on='id')
del card_zhifu
'''

## consume feature
consume=card_train_test[card_train_test.pos == 'POS消费']

card_consume = pd.DataFrame(consume.groupby(['id'])['pos'].count())

card_consume['consume_sum']=consume.groupby(['id'])['price'].sum()
card_consume['consume_avg']=consume.groupby(['id'])['price'].mean()
card_consume['consume_max']=consume.groupby(['id'])['price'].max()
card_consume['consume_min']=consume.groupby(['id'])['price'].min()
card_consume['consume_median']=consume.groupby(['id'])['price'].median()

del consume
card_consume.to_csv('input/card_consumefeature.csv',index=True)
card_consume = pd.read_csv('input/card_consumefeature.csv') 
card_consume=card_consume.rename(columns={'pos' : 'consume_count'}) 


train_test = pd.merge(train_test, card_consume, how='left',on='id') 
del card_consume

kapiankaihu=card_train_test[card_train_test.pos=='卡片开户']
card_kaihu = pd.DataFrame(kapiankaihu.groupby(['id'])['pos'].count())
del kapiankaihu
card_kaihu.to_csv('input/card_kaihufeature.csv',index=True)
card_kaihu = pd.read_csv('input/card_kaihufeature.csv') 
card_kaihu=card_kaihu.rename(columns={'pos' : 'kaihu_count'}) 

train_test = pd.merge(train_test, card_kaihu, how='left',on='id') 
del card_kaihu

kapianxiaohu=card_train_test[card_train_test.pos=='卡片销户']
card_xiaohu = pd.DataFrame(kapianxiaohu.groupby(['id'])['pos'].count())
del kapianxiaohu
card_xiaohu.to_csv('input/card_xiaohufeature.csv',index=True)
card_xiaohu = pd.read_csv('input/card_xiaohufeature.csv') 
card_xiaohu=card_xiaohu.rename(columns={'pos' : 'xiaohu_count'}) 

train_test = pd.merge(train_test, card_xiaohu, how='left',on='id') 
del card_xiaohu

kapianbuban=card_train_test[card_train_test.pos=='卡补办']
card_buban = pd.DataFrame(kapianbuban.groupby(['id'])['pos'].count())
del kapianbuban
card_buban.to_csv('input/card_bubanfeature.csv',index=True)
card_buban = pd.read_csv('input/card_bubanfeature.csv') 
card_buban=card_buban.rename(columns={'pos' : 'buban_count'}) 

train_test = pd.merge(train_test, card_buban, how='left',on='id') 
del card_buban

kapianjiegua=card_train_test[card_train_test.pos=='卡解挂']
card_jiegua = pd.DataFrame(kapianjiegua.groupby(['id'])['pos'].count())
del kapianjiegua
card_jiegua.to_csv('input/card_jieguafeature.csv',index=True)
card_jiegua = pd.read_csv('input/card_jieguafeature.csv') 
card_jiegua=card_jiegua.rename(columns={'pos' : 'jiegua_count'}) 

train_test = pd.merge(train_test, card_jiegua, how='left',on='id') 
del card_jiegua

kapianchange=card_train_test[card_train_test.pos=='换卡']
card_change = pd.DataFrame(kapianchange.groupby(['id'])['pos'].count())
del kapianchange
card_change.to_csv('input/card_changefeature.csv',index=True)
card_change = pd.read_csv('input/card_changefeature.csv') 
card_change=card_change.rename(columns={'pos' : 'change_count'}) 

train_test = pd.merge(train_test, card_change, how='left',on='id') 
del card_change



canteen=card_train_test[card_train_test.consume=='食堂']

card_canteen = pd.DataFrame(canteen.groupby(['id'])['pos'].count())
card_canteen['canteen_sum']=canteen.groupby(['id'])['price'].sum()
card_canteen['canteen_avg']=canteen.groupby(['id'])['price'].mean()
card_canteen['canteen_max']=canteen.groupby(['id'])['price'].max()
card_canteen['canteen_min']=canteen.groupby(['id'])['price'].min()
card_canteen['canteen_median']=canteen.groupby(['id'])['price'].median()

del canteen
card_canteen.to_csv('input/card_canteenfeature.csv',index=True)
card_canteen = pd.read_csv('input/card_canteenfeature.csv') 
card_canteen=card_canteen.rename(columns={'pos' : 'canteen_count'}) 

train_test = pd.merge(train_test, card_canteen, how='left',on='id') 
del card_canteen

boiled_water=card_train_test[card_train_test.consume=='开水']

card_boiled_water = pd.DataFrame(boiled_water.groupby(['id'])['pos'].count())
card_boiled_water['boiled_water_sum']=boiled_water.groupby(['id'])['price'].sum()
card_boiled_water['boiled_water_avg']=boiled_water.groupby(['id'])['price'].mean()
card_boiled_water['boiled_water_max']=boiled_water.groupby(['id'])['price'].max()
card_boiled_water['boiled_water_min']=boiled_water.groupby(['id'])['price'].min()
card_boiled_water['boiled_water_median']=boiled_water.groupby(['id'])['price'].median()

del boiled_water
card_boiled_water.to_csv('input/card_boiled_waterfeature.csv',index=True)
card_boiled_water = pd.read_csv('input/card_boiled_waterfeature.csv') 
card_boiled_water=card_boiled_water.rename(columns={'pos' : 'boiled_water_count'}) 

train_test = pd.merge(train_test, card_boiled_water, how='left',on='id') 
del card_boiled_water

bathe=card_train_test[card_train_test.consume=='淋浴']

card_bathe = pd.DataFrame(bathe.groupby(['id'])['pos'].count())
card_bathe['bathe_sum']=bathe.groupby(['id'])['price'].sum()
card_bathe['bathe_avg']=bathe.groupby(['id'])['price'].mean()
card_bathe['bathe_max']=bathe.groupby(['id'])['price'].max()
card_bathe['bathe_min']=bathe.groupby(['id'])['price'].min()
card_bathe['bathe_median']=bathe.groupby(['id'])['price'].median()

del bathe
card_bathe.to_csv('input/card_bathefeature.csv',index=True)
card_bathe = pd.read_csv('input/card_bathefeature.csv') 
card_bathe=card_bathe.rename(columns={'pos' : 'bathe_count'}) 

train_test = pd.merge(train_test, card_bathe, how='left',on='id') 
del card_bathe

shool_bus=card_train_test[card_train_test.consume=='校车']

card_shool_bus = pd.DataFrame(shool_bus.groupby(['id'])['pos'].count())
card_shool_bus['shool_bus_sum']=shool_bus.groupby(['id'])['price'].sum()
card_shool_bus['shool_bus_avg']=shool_bus.groupby(['id'])['price'].mean()
card_shool_bus['shool_bus_max']=shool_bus.groupby(['id'])['price'].max()
card_shool_bus['shool_bus_min']=shool_bus.groupby(['id'])['price'].min()
card_shool_bus['shool_bus_median']=shool_bus.groupby(['id'])['price'].median()

del shool_bus
card_shool_bus.to_csv('input/card_shool_busfeature.csv',index=True)
card_shool_bus = pd.read_csv('input/card_shool_busfeature.csv') 
card_shool_bus=card_shool_bus.rename(columns={'pos' : 'shool_bus_count'}) 

train_test = pd.merge(train_test, card_shool_bus, how='left',on='id') 
del card_shool_bus

shop=card_train_test[card_train_test.consume=='超市']

card_shop = pd.DataFrame(shop.groupby(['id'])['pos'].count())
card_shop['shop_sum']=shop.groupby(['id'])['price'].sum()
card_shop['shop_avg']=shop.groupby(['id'])['price'].mean()
card_shop['shop_max']=shop.groupby(['id'])['price'].max()
card_shop['shop_min']=shop.groupby(['id'])['price'].min()
card_shop['shop_median']=shop.groupby(['id'])['price'].median()

del shop
card_shop.to_csv('input/card_shopfeature.csv',index=True)
card_shop = pd.read_csv('input/card_shopfeature.csv') 
card_shop=card_shop.rename(columns={'pos' : 'shop_count'}) 

train_test = pd.merge(train_test, card_shop, how='left',on='id') 
del card_shop

wash_house=card_train_test[card_train_test.consume=='洗衣房']

card_wash_house = pd.DataFrame(wash_house.groupby(['id'])['pos'].count())
card_wash_house['wash_sum']=wash_house.groupby(['id'])['price'].sum()
card_wash_house['wash_avg']=wash_house.groupby(['id'])['price'].mean()
card_wash_house['wash_max']=wash_house.groupby(['id'])['price'].max()
card_wash_house['wash_min']=wash_house.groupby(['id'])['price'].min()
card_wash_house['wash_median']=wash_house.groupby(['id'])['price'].median()

del wash_house
card_wash_house.to_csv('input/card_wash_housefeature.csv',index=True)
card_wash_house = pd.read_csv('input/card_wash_housefeature.csv') 
card_wash_house=card_wash_house.rename(columns={'pos' : 'wash_house_count'}) 

train_test = pd.merge(train_test, card_wash_house, how='left',on='id') 
del card_wash_house

library=card_train_test[card_train_test.consume=='图书馆']

card_library= pd.DataFrame(library.groupby(['id'])['pos'].count())
card_library['library_sum']=library.groupby(['id'])['price'].sum()
card_library['library_avg']=library.groupby(['id'])['price'].mean()
card_library['library_max']=library.groupby(['id'])['price'].max()
card_library['library_min']=library.groupby(['id'])['price'].min()
card_library['library_median']=library.groupby(['id'])['price'].median()

del library
card_library.to_csv('input/card_libraryfeature.csv',index=True)
card_library = pd.read_csv('input/card_libraryfeature.csv') 
card_library=card_library.rename(columns={'pos' : 'library_count'}) 

train_test = pd.merge(train_test, card_library, how='left',on='id') 
del card_library

printhouse=card_train_test[card_train_test.consume=='文印中心']

card_printhouse= pd.DataFrame(printhouse.groupby(['id'])['pos'].count())
card_printhouse['print_sum']=printhouse.groupby(['id'])['price'].sum()
card_printhouse['print_avg']=printhouse.groupby(['id'])['price'].mean()
card_printhouse['print_max']=printhouse.groupby(['id'])['price'].max()
card_printhouse['print_min']=printhouse.groupby(['id'])['price'].min()
card_printhouse['print_median']=printhouse.groupby(['id'])['price'].median()

del printhouse
card_printhouse.to_csv('input/card_printhousefeature.csv',index=True)
card_printhouse = pd.read_csv('input/card_printhousefeature.csv') 
card_printhouse=card_printhouse.rename(columns={'pos' : 'printhouse_count'}) 

train_test = pd.merge(train_test, card_printhouse, how='left',on='id') 
del card_printhouse

dean=card_train_test[card_train_test.consume=='教务处']

card_dean= pd.DataFrame(dean.groupby(['id'])['pos'].count())
card_dean['dean_sum']=dean.groupby(['id'])['price'].sum()
card_dean['dean_avg']=dean.groupby(['id'])['price'].mean()
card_dean['dean_max']=dean.groupby(['id'])['price'].max()
card_dean['dean_min']=dean.groupby(['id'])['price'].min()
card_dean['dean_median']=dean.groupby(['id'])['price'].median()

del dean
card_dean.to_csv('input/card_deanfeature.csv',index=True)
card_dean = pd.read_csv('input/card_deanfeature.csv') 
card_dean=card_dean.rename(columns={'pos' : 'dean_count'}) 

train_test = pd.merge(train_test, card_dean, how='left',on='id') 
del card_dean

other=card_train_test[card_train_test.consume=='其他']

card_other= pd.DataFrame(other.groupby(['id'])['pos'].count())
card_other['other_sum']=other.groupby(['id'])['price'].sum()
#
card_other['other_avg']=other.groupby(['id'])['price'].mean()
#
card_other['other_max']=other.groupby(['id'])['price'].max()
#
card_other['other_min']=other.groupby(['id'])['price'].min()
#
card_other['other_median']=other.groupby(['id'])['price'].median()

del other
card_other.to_csv('input/card_otherfeature.csv',index=True)
card_other = pd.read_csv('input/card_otherfeature.csv') 
card_other=card_other.rename(columns={'pos' : 'other_count'}) 

train_test = pd.merge(train_test, card_other, how='left',on='id') 
del card_other


hospital=card_train_test[card_train_test.consume=='校医院']

card_hospital= pd.DataFrame(hospital.groupby(['id'])['pos'].count())
card_hospital['hospital_sum']=hospital.groupby(['id'])['price'].sum()
card_hospital['hospital_avg']=hospital.groupby(['id'])['price'].mean()
card_hospital['hospital_max']=hospital.groupby(['id'])['price'].max()
card_hospital['hospital_min']=hospital.groupby(['id'])['price'].min()
card_hospital['hospital_median']=hospital.groupby(['id'])['price'].median()

del hospital
card_hospital.to_csv('input/card_hospitalfeature.csv',index=True)
card_hospital = pd.read_csv('input/card_hospitalfeature.csv') 
card_hospital=card_hospital.rename(columns={'pos' : 'hospital_count'}) 

train_test = pd.merge(train_test, card_hospital, how='left',on='id') 
del card_hospital

In [3]:
train_test=pd.read_csv('input/train_time.csv')

In [4]:
#del card_train_test
train_test=train_test.fillna(-1)

In [5]:
train_test.head()

,id,label,college,rank,total_people,rank_percent,countM1,price_sumM1,price_avgM1,price_maxM1,...,地点263_avg,地点263_max,地点263_min,地点263_median,地点840_count,地点840_sum,地点840_avg,地点840_max,地点840_min,地点840_median
0,0,0.0,9.0,1.0,2933.0,0.000341,49.0,201.31,4.108367,36.4,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.000000,-1.0,-1.0,-1.0
1,1,0.0,9.0,2.0,2933.0,0.000682,-1.0,-1.00,-1.000000,-1.0,...,-1.0,-1.0,-1.0,-1.0,19.0,117.0,6.157895,7.0,4.0,6.0
2,8,0.0,6.0,1565.0,1570.0,0.996815,97.0,347.74,3.584948,10.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.000000,-1.0,-1.0,-1.0
3,9,0.0,6.0,1570.0,1570.0,1.000000,98.0,491.01,5.010306,17.5,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.000000,-1.0,-1.0,-1.0
4,10,0.0,3.0,1.0,2304.0,0.000434,27.0,82.96,3.072593,22.3,...,-1.0,-1.0,-1.0,-1.0,2.0,7.0,3.500000,4.0,3.0,3.5


In [6]:
## Evaluation 
def f1_macro(label_truth, predictions):
    df=pd.DataFrame(columns=["subsidy_x","subsidy_y"])
    df.subsidy_y=predictions
    df.subsidy_x=np.array(label_truth)
    df.subsidy_y = df.subsidy_y.apply(lambda x:int(x))

    
    correct = df[df['subsidy_x'] == df['subsidy_y']]
    s = 0
    for i in [1000, 1500, 2000]:
        print '\n%d'%i
        if sum(df['subsidy_x'] == i)!=0:
            r = float(sum(correct['subsidy_y'] == i))/sum(df['subsidy_x'] == i)
            print 'Recall---%s'%r
        else: 
            r=0
        if sum(df['subsidy_y'] == i)!=0:
            p = float(sum(correct['subsidy_y'] == i))/sum(df['subsidy_y'] == i)        
            print 'Precision---%s'%p
        else:
            p=0
        if (r+p)!=0:
            f = r*p*2/(r+p)
            print 'F1---%s'%f
        if not np.isnan(f):
            s += (float(sum(df['subsidy_x'] == i))/df.shape[0])*f
    
    print '\nF1-macro---%s'%s
    return s

def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

In [ ]:
nice_feature=pd.read_csv('input/nice_feature.csv',header=None,index_col=0)

target = 'label'
IDcol = 'id'
ids = test['id'].values

all_feature = [x for x in train.columns if x not in [target]]

#predictors = [ x for x in all_feature if x in all_feature]
#predictors = [ x for x in all_feature if x in nice_feature.index]

feature_imp_place20=pd.read_csv('input/feature_imp_place20.csv')

predictors = [ x for x in all_feature if (x in nice_feature.index)|(x in feature_imp_place20.feature.values)]

In [7]:
X = train_test.ix[:, train_test.columns != 'label']
y = train_test.ix[:, train_test.columns == 'label']

In [93]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size = 0.3, random_state = 7)

In [94]:
train=X_train
train['label']=y_train

test=X_test

y_train['label'].value_counts()


# tips：
# 尝试欠采样和smoth采样

# undersample
train.to_csv('input/train_7r07part.csv',index=False)
train=pd.read_csv('input/train_7r07part.csv')

number_records_fraud = len(train[train.label != 0])
fraud_indices = np.array(train[train.label != 0].index)

# Picking the indices of the normal classes
normal_indices = train[train.label == 0].index

# Out of the indices we picked, randomly select "x" number (number_records_fraud)
random_normal_indices = np.random.choice(normal_indices, number_records_fraud, replace = False)
random_normal_indices = np.array(random_normal_indices)

# Appending the 2 indices
under_sample_indices = np.concatenate([fraud_indices,random_normal_indices])

# Under sample dataset
under_sample_data = train.iloc[under_sample_indices,:]

X_undersample = under_sample_data.ix[:, under_sample_data.columns != 'label']
y_undersample = under_sample_data.ix[:, under_sample_data.columns == 'label']

# Showing ratio
print("Number of normal transactions: ", len(under_sample_data[under_sample_data.label == 0]))
print("Number of fraud transactions: ", len(under_sample_data[under_sample_data.label != 0]))
print("Total number of transactions in resampled data: ", len(under_sample_data))


# Oversample
Oversampling1000 = train.loc[train.label == 1000]
Oversampling1500 = train.loc[train.label == 1500]
Oversampling2000 = train.loc[train.label == 2000]

for i in range(5):
    train = train.append(Oversampling1000)
for j in range(7):
    train = train.append(Oversampling1500)
for k in range(9):
    train = train.append(Oversampling2000)
'''
for i in range(5):
    train = train.append(Oversampling1000)
for j in range(8):
    train = train.append(Oversampling1500)
for k in range(10):
    train = train.append(Oversampling2000)

'''   
train = train[train['label'].notnull()]

/home/kuhung/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


('Number of normal transactions: ', 1089)
('Number of fraud transactions: ', 1089)
('Total number of transactions in resampled data: ', 2178)


In [95]:
train.shape

(14690, 432)

In [96]:
clf=XGBClassifier(max_depth=4,objective='multi:softmax',n_estimators=100,seed=42)
clf = clf.fit(train[predictors],train[target])
result = clf.predict(test[predictors])

f1_macro(y_test,result)


1000
Recall---0.380530973451
Precision---0.19153674833
F1---0.254814814815

1500
Recall---0.215384615385
Precision---0.141414141414
F1---0.170731707317

2000
Recall---0.208695652174
Precision---0.140350877193
F1---0.167832167832

F1-macro---0.0303380187998


0.030338018799775556

In [ ]:

1000
Recall---0.544247787611
Precision---0.188940092166
F1---0.280501710376

1500
Recall---0.207692307692
Precision---0.151685393258
F1---0.175324675325

2000
Recall---0.191304347826
Precision---0.164179104478
F1---0.176706827309

F1-macro---0.0326108020446


In [ ]:
from sklearn import preprocessing

r_scale=preprocessing.RobustScaler()

train_scaled=r_scale.fit_transform(train[predictors])
test_scaled=r_scale.transform(test[predictors])

from sklearn.metrics import f1_score

# model
# 对照组
clf=XGBClassifier(max_depth=4,objective='multi:softmax',n_estimators=100,seed=42)
clf = clf.fit(train[predictors],train[target])
result = clf.predict(test[predictors])
print '\n--------XGBClassifier--------'
f1_macro(y_test,result)

clf=XGBClassifier(max_depth=4,objective='multi:softmax',n_estimators=180,seed=42,learning_rate=0.05)
clf = clf.fit(train[predictors],train[target])
result = clf.predict(test[predictors])
print '\n--------XGBClassifier(learning_rate=0.05)--------'
f1_macro(y_test,result)

clf=XGBClassifier(max_depth=4,objective='multi:softmax',n_estimators=100,seed=42)
clf = clf.fit(X_undersample[predictors],y_undersample[target])
result = clf.predict(test[predictors])
print '\n--------XGBClassifier(undersample)--------'
f1_macro(y_test,result)


clf=XGBClassifier(max_depth=4,objective='multi:softmax',n_estimators=100,seed=42)
clf = clf.fit(np.log(train[predictors]),train[target])
result = clf.predict(np.log(test[predictors]))
print '\n--------XGBClassifier(log)--------'
f1_macro(y_test,result)


clf=XGBClassifier(max_depth=4,objective='multi:softmax',n_estimators=100,seed=42)
clf = clf.fit(train_scaled,train[target])
result = clf.predict(test_scaled)
print '\n--------XGBClassifier(scaled)--------'
f1_macro(y_test,result)

# model
#

clf = GradientBoostingClassifier(random_state=42,n_estimators=200)
clf = clf.fit(train[predictors],train[target])
result = clf.predict(test[predictors])
print '\n--------GradientBoosting--------'
f1_macro(y_test,result)

clf = GradientBoostingClassifier(random_state=42,n_estimators=200)
clf = clf.fit(X_undersample[predictors],y_undersample[target])
result = clf.predict(test[predictors])
print '\n--------GradientBoosting(undersample)--------'
f1_macro(y_test,result)

# GBC log会报错
#clf = GradientBoostingClassifier(random_state=42,n_estimators=200)
#clf = clf.fit(np.log(train[predictors]),train[target])
#result = clf.predict(np.log(test[predictors]))
#print '\n--------GradientBoosting(log)--------'
#f1_macro(y_test,result)

clf = GradientBoostingClassifier(random_state=42,n_estimators=200)
clf = clf.fit(train_scaled,train[target])
result = clf.predict(test_scaled)
print '\n--------GradientBoosting(scaled)--------'
f1_macro(y_test,result)



# 
clf = RandomForestClassifier()
clf = clf.fit(train[predictors],train[target])
result = clf.predict(test[predictors])
print '\n--------RFClassifier--------'
f1_macro(y_test,result)


clf = RandomForestClassifier()
clf = clf.fit(X_undersample[predictors],y_undersample[target])
result = clf.predict(test[predictors])
print '\n--------RFClassifier(undersample)--------'
f1_macro(y_test,result)

# log 报错
#clf = RandomForestClassifier()
#clf = clf.fit(np.log(train[predictors]),train[target])
#result = clf.predict(np.log(test[predictors]))
#print '\n--------RFClassifier(log)--------'
#f1_macro(y_test,result)

clf = RandomForestClassifier()
clf = clf.fit(train_scaled,train[target])
result = clf.predict(test_scaled)
print '\n--------RFClassifier(scaled)--------'
f1_macro(y_test,result)


#f1_macro(y_test,result)
#plot_confusion_matrix(confusion_matrix(y_test,result),classes=[0,1000,1500,2000])

# tips：
# 尝试用二分类（得与不得，得多少的分类）
# 尝试神经网络
# 模型调参(常规参数与阈值)





In [18]:
# 由于聚类的精度不如逻辑回归二分类，所以todo is
# 将结果进行二分类后，再对分类结果为正的进行训练。

In [19]:
'''
# else trick

def model_f1_macro(name,clf):
    clf=clf
    clf = clf.fit(train[predictors],train[target])
    result = clf.predict(test[predictors])
    print '\n--------%s--------'%name
    f1_macro(y_test,result)
    
from sklearn.cross_validation import cross_val_score

# meta-estimator
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier 

from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis


classifiers = {
    'KN': KNeighborsClassifier(3),
    'DT': DecisionTreeClassifier(max_depth=5),
    'RF': RandomForestClassifier(n_estimators=10, max_depth=5, max_features=1),  # clf.feature_importances_
    'ET': ExtraTreesClassifier(n_estimators=10, max_depth=None),  # clf.feature_importances_
    'AB': AdaBoostClassifier(n_estimators=100),
    'GB': GradientBoostingClassifier(n_estimators=100, learning_rate=1.0, max_depth=1, random_state=0), # clf.feature_importances_
    'GNB': GaussianNB(),
    'LD': LinearDiscriminantAnalysis(),
    'QD': QuadraticDiscriminantAnalysis()}

    
    
X, y = train[predictors],train[target]

for name, clf in classifiers.items():
    scores = cross_val_score(clf, X, y)
    print '\n',name,'\t--> ',scores.mean()
    model_f1_macro(name,clf)
    
'''

"\n# else trick\n\ndef model_f1_macro(name,clf):\n    clf=clf\n    clf = clf.fit(train[predictors],train[target])\n    result = clf.predict(test[predictors])\n    print '\n--------%s--------'%name\n    f1_macro(y_test,result)\n    \nfrom sklearn.cross_validation import cross_val_score\n\n# meta-estimator\nfrom sklearn.neighbors import KNeighborsClassifier\nfrom sklearn.svm import SVC\nfrom sklearn.tree import DecisionTreeClassifier\nfrom sklearn.ensemble import RandomForestClassifier\nfrom sklearn.ensemble import ExtraTreesClassifier\nfrom sklearn.ensemble import AdaBoostClassifier\nfrom sklearn.ensemble import GradientBoostingClassifier \n\nfrom sklearn.naive_bayes import GaussianNB\nfrom sklearn.discriminant_analysis import LinearDiscriminantAnalysis\nfrom sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis\n\n\nclassifiers = {\n    'KN': KNeighborsClassifier(3),\n    'DT': DecisionTreeClassifier(max_depth=5),\n    'RF': RandomForestClassifier(n_estimators=10, max_dept

In [20]:

'''
## vote
from sklearn import ensemble

clf1 = XGBClassifier(max_depth=4,objective='multi:softmax',n_estimators=100,seed=42)
clf2 = XGBClassifier(max_depth=3,objective='multi:softmax',n_estimators=100,seed=42)
#lf3 = XGBClassifier(max_depth=4,objective='multi:softmax',n_estimators=100,seed=0)
#clf4 = XGBClassifier(max_depth=3,objective='multi:softmax',n_estimators=100,seed=0)

clf3 = GradientBoostingClassifier(random_state=42)
clf4 = GradientBoostingClassifier(n_estimators=200,random_state=42)

clfs=ensemble.VotingClassifier(estimators=[('xgb1',clf1),('xgb2',clf2),('GBM',clf3),('RF',clf4)],voting='hard')
clfs = clfs.fit(train[predictors],train[target])
#clfs = clfs.fit(X_undersample[predictors],y_undersample[target])

result = clfs.predict(test[predictors])
print '\n--------Vote--------'
f1_macro(y_test,result)

'''

"\n## vote\nfrom sklearn import ensemble\n\nclf1 = XGBClassifier(max_depth=4,objective='multi:softmax',n_estimators=100,seed=42)\nclf2 = XGBClassifier(max_depth=3,objective='multi:softmax',n_estimators=100,seed=42)\n#lf3 = XGBClassifier(max_depth=4,objective='multi:softmax',n_estimators=100,seed=0)\n#clf4 = XGBClassifier(max_depth=3,objective='multi:softmax',n_estimators=100,seed=0)\n\nclf3 = GradientBoostingClassifier(random_state=42)\nclf4 = GradientBoostingClassifier(n_estimators=200,random_state=42)\n\nclfs=ensemble.VotingClassifier(estimators=[('xgb1',clf1),('xgb2',clf2),('GBM',clf3),('RF',clf4)],voting='hard')\nclfs = clfs.fit(train[predictors],train[target])\n#clfs = clfs.fit(X_undersample[predictors],y_undersample[target])\n\nresult = clfs.predict(test[predictors])\nprint '\n--------Vote--------'\nf1_macro(y_test,result)\n\n"

In [21]:
'''
from sklearn.pipeline import make_pipeline
from sklearn.decomposition import PCA

pca=PCA(n_components=100)

clf=make_pipeline(pca,GradientBoostingClassifier())

clf = clf.fit(train[predictors],train[target])
result = clf.predict(test[predictors])
f1_macro(y_test,result)
'''

'\nfrom sklearn.pipeline import make_pipeline\nfrom sklearn.decomposition import PCA\n\npca=PCA(n_components=100)\n\nclf=make_pipeline(pca,GradientBoostingClassifier())\n\nclf = clf.fit(train[predictors],train[target])\nresult = clf.predict(test[predictors])\nf1_macro(y_test,result)\n'

In [22]:
'''
## out点处理
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

clf=make_pipeline(StandardScaler(),GradientBoostingClassifier())

clf = clf.fit(train[predictors],train[target])
result = clf.predict(test[predictors])
f1_macro(y_test,result)

'''

'\n## out\xe7\x82\xb9\xe5\xa4\x84\xe7\x90\x86\nfrom sklearn.pipeline import make_pipeline\nfrom sklearn.preprocessing import StandardScaler\n\nclf=make_pipeline(StandardScaler(),GradientBoostingClassifier())\n\nclf = clf.fit(train[predictors],train[target])\nresult = clf.predict(test[predictors])\nf1_macro(y_test,result)\n\n'

In [23]:
'''
def score(y_true,y_pred):
    y_true=pd.Series(y_true)
    y_pred=pd.Series(y_pred)
    count=y_true.value_counts()
    
    #y_pred=pd.Series(y_pred)
    #count=y_true[target].value_counts()
    
    score1000=f1_score(y_true==1000,y_pred==1000)*count[1000]/y_true.size
    score1500=f1_score(y_true==1500,y_pred==1500)*count[1500]/y_true.size
    score2000=f1_score(y_true==2000,y_pred==2000)*count[2000]/y_true.size
    
    return score1000+score1500+score2000
    
'''

'\ndef score(y_true,y_pred):\n    y_true=pd.Series(y_true)\n    y_pred=pd.Series(y_pred)\n    count=y_true.value_counts()\n    \n    #y_pred=pd.Series(y_pred)\n    #count=y_true[target].value_counts()\n    \n    score1000=f1_score(y_true==1000,y_pred==1000)*count[1000]/y_true.size\n    score1500=f1_score(y_true==1500,y_pred==1500)*count[1500]/y_true.size\n    score2000=f1_score(y_true==2000,y_pred==2000)*count[2000]/y_true.size\n    \n    return score1000+score1500+score2000\n    \n'

In [24]:
def f1_macro0_drop(label_truth, predictions):
    df=pd.DataFrame(columns=["subsidy_x","subsidy_y"])
    df.subsidy_y=predictions
    df.subsidy_x=np.array(label_truth)
    df.subsidy_y = df.subsidy_y.apply(lambda x:int(x))

    
    correct = df[df['subsidy_x'] == df['subsidy_y']]
    s = 0
    for i in [1000, 1500, 2000]:
        if sum(df['subsidy_x'] == i)!=0:
            r = float(sum(correct['subsidy_y'] == i))/sum(df['subsidy_x'] == i)
        else: 
            r=0
        if sum(df['subsidy_y'] == i)!=0:
            p = float(sum(correct['subsidy_y'] == i))/sum(df['subsidy_y'] == i)        
        else:
            p=0
        if (r+p)!=0:
            f = r*p*2/(r+p)
        if not np.isnan(f):
            s += (float(sum(df['subsidy_x'] == i))/df.shape[0])*f
    
    return s

In [25]:
from sklearn.metrics import make_scorer

my_score=make_scorer(score_func=f1_macro0_drop)

In [26]:
# 参数调节

from sklearn import cross_validation, metrics  
from sklearn.model_selection import GridSearchCV 

def modelfit(alg, dtrain, predictors, target,performCV=True, printFeatureImportance=False, cv_folds=5):
    #Fit the algorithm on the data
    alg.fit(dtrain[predictors], dtrain[target])

    #Predict training set:
    dtrain_predictions = alg.predict(dtrain[predictors])
    dtrain_predprob = alg.predict_proba(dtrain[predictors])[:,1]

    #Perform cross-validation:
    if performCV:
        cv_score = cross_validation.cross_val_score(alg, dtrain[predictors], dtrain[target], cv=cv_folds, scoring=my_score)

    #Print model report:
    print "\nModel Report"
    print "Accuracy : %.4g" % metrics.accuracy_score(dtrain[target].values, dtrain_predictions)
    
    if performCV:
        print "CV Score : Mean - %.7g | Std - %.7g | Min - %.7g | Max - %.7g" % (np.mean(cv_score),np.std(cv_score),np.min(cv_score),np.max(cv_score))

    #Print Feature Importance:
    if printFeatureImportance:
        feat_imp = pd.Series(alg.feature_importances_, predictors).sort_values(ascending=False)
        feat_imp.plot(kind='bar', title='Feature Importances')
        plt.ylabel('Feature Importance Score')



/home/kuhung/anaconda2/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [27]:
gbm0 = GradientBoostingClassifier(random_state=10)
modelfit(gbm0, train, predictors,target)


Model Report
Accuracy : 0.8567
CV Score : Mean - 0.4730227 | Std - 0.005252156 | Min - 0.4655394 | Max - 0.4794882


In [28]:
## why the score of it is higher?
## Because of resample.After resample,the dis of dataset had changed.
# 0.02779836483035414

xgb0 = XGBClassifier(max_depth=4,objective='multi:softmax',n_estimators=100,seed=42)
modelfit(xgb0, under_sample_data, predictors,target)

param_test1 = {'n_estimators':range(20,81,10)}
gsearch1 = GridSearchCV(estimator = GradientBoostingClassifier(learning_rate=0.1, min_samples_split=80,min_samples_leaf=8,max_depth=3,max_features='sqrt',subsample=0.8,random_state=10), 
param_grid = param_test1, scoring=my_score,n_jobs=4,iid=False, cv=5)
#gsearch1.fit(train[predictors],train[target])
gsearch1.fit(X_undersample,y_undersample)

gsearch1.grid_scores_, gsearch1.best_params_, gsearch1.best_score_



Model Report
Accuracy : 0.8737
CV Score : Mean - 0.1572691 | Std - 0.01077795 | Min - 0.1430099 | Max - 0.1729337


IndexError: too many indices for array

In [ ]:
## Feature importance
'''
clf=XGBClassifier(max_depth=4,objective='multi:softmax',n_estimators=100,seed=42)
clf = clf.fit(train[predictors],train[target])
result = clf.predict_proba(test[predictors])
f1_macro(y_test,result)

feat_imp=pd.Series(clf.booster().get_fscore()).sort_values(ascending=False)
feat_imp.plot(kind='bar', title='Feature Importances')
feat_imp.tail(50)

'''

In [ ]:
'''
## Evaluation
def score(df):
    # df有三列，ID:学生ID,subsidy_x:实际奖学金金额,subsidy_y:预测奖学金金额
    correct = test_result[test_result['subsidy_x'] == test_result['subsidy_y']]
    s = 0
    for i in [1000, 1500, 2000]:
        r = float(sum(correct['subsidy_y'] == i))/sum(test_result['subsidy_x'] == i)
        p = float(sum(correct['subsidy_y'] == i))/sum(test_result['subsidy_y'] == i)
        f = r*p*2/(r+p)
        if not np.isnan(f):
            s += (float(sum(test_result['subsidy_x'] == i))/test_result.shape[0])*f
    print(s)

test_result = pd.DataFrame(columns=["studentid","subsidy_x","subsidy_y"])
test_result.studentid = ids

test_result.subsidy_x =np.array(y_test)
test_result.subsidy_y = result
test_result.subsidy_y = test_result.subsidy_y.apply(lambda x:int(x))

score(test_result)

'''